## Selection statement with more than two selection blocks

Try to implement a workflow in which all the operators of the selection interface *(OPH_IF, OPH_ELSEIF, OPH_ELSE, OPH_ENDIF)* are adopted.

The following figure shows the graphical representation of a possible workflow.

<img src="../imgs/Selection_Interface.svg" alt="Selection_Interface" width="300">

The corresponding pseudo-code could be something like this:

    if flag1 == 1:
        Import a NetCDF file and subset it at the same time
    else 
        if flag2 == 1:
            Create a container and then a new datacube with random data and dimensions
        else
            Import a NetCDF file, then perform a subset operation over the datacube dimensions

As first step, connect to the Ophidia Server

In [ ]:
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

Let's try to build step-by-step the JSON object associated to the workflow, by completing the missing arguments (```.....```).

**Some hints:**
   - the task ```OPH_ELSEIF``` has to be a child of ```OPH_IF``` or another ```OPH_ELSEIF```
   - the task ```OPH_ELSE``` could be a child of ```OPH_ELSEIF``` and must be child of ```OPH_IF```
   - the last task of any branch should be added as a dependency of the ```OPH_ENDIF``` task
   - the workflow has the following arguments:
       -  *if condition* ```$1```
       -  *elseif condition* ```$2```
       -  *NetCDF filename* ```$3```
       -  *variable* to be imported ```$4```
       -  *subset filter* (lat|lon|time) ```$5```
 

First of all, we need to define the global workflow information.

In [ ]:
workflow_if_endif_else = """{
        "name": "Selection Interface",
        "author": "CMCC",
        "abstract": "Selection statement with more than two selection blocks",
        "exec_mode": "sync",
        "ncores": "1",
        "tasks":
        [
"""

**IF block**

Open a ```if``` selection block and start the first selection sub-block.

The boolean expression to be checked is the first argument of the workflow: if ```true``` (a *numerical non-zero value*), the tasks depending on this task are executed, whereas the tasks associated with the following selection sub-blocks are skipped; if ```false``` (a *zero value*), the tasks depending on this task are skipped, whereas the tasks associated with the following selection sub-block are executed.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "IF block",
                        "operator": ".....",
                        "arguments": [ "condition=$1" ]
                },
"""

If the first condition is ```true```, use the ```oph_importnc``` operator to import and subset a NetCDF file at the same time. Note use the workflow variables ```$N``` for the import operator arguments.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "Import and subset",
                        "operator": ".....",
                        "arguments":
                        [
                                "src_path=.....",
                                "measure=.....",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp imported and subsetted",
                                "subset_dims=lat|lon|time",
                                "subset_filter=.....",
                                "subset_type=coord"
                        ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },

"""

Start a new sub-block of the selection block ```if```. Here the boolean expression to be checked is the second argument of the workflow.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "ELSEIF block",
                        "operator": "oph_elseif",
                        "arguments": [ "condition=....." ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },

"""

If the second condition is ```true```, create an empty container named ```workflow``` and then a new datacube with random data and dimensions.

In [ ]:
workflow_if_endif_else += """
               {
                        "name": "Create container",
                        "operator": "oph_createcontainer",
                        "on_error": "skip",
                        "arguments": 
                        [
                                "container=random",
                                "dim=lat|lon|time",
                                "dim_type=double|double|double",
                                "hierarchy=oph_base|oph_base|oph_time"
                        ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },
                {
                        "name": "Randcube",
                        "operator": "oph_randcube",
                        "arguments":
                        [
                                "container=random",
                                "measure_type=double",
                                "exp_ndim=2",
                                "dim=lat|lon|time",
                                "concept_level=c|c|d",
                                "dim_size=16|10|10",
                                "compressed=yes",
                                "measure=test_var",
                                "description=random cube",
                                "nhost=1",
                                "nfrag=16",
                                "ntuple=10"
                        ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },

"""

Start the last sub-block of the selection block ```if```.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "ELSE block",
                        "operator": "oph_else",
                        "arguments": [  ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },

"""

If neither of the two conditions is satisfied, first import a NetCDF file, then perform a subset operation over the datacube dimensions.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "Import data",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=.....",
                                "measure=.....",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp imported"
                        ],
                        "dependencies":
                        [
                                { "task": "....." }
                        ]
                },
                {
                        "name": "Subset data",
                        "operator": "oph_subset",
                        "arguments":
                        [
                               "subset_dims=.....",
                               "subset_filter=.....",
                               "subset_type=coord",
                               "description=Max Temp subsetted" 
                        ],
                        "dependencies":
                        [
                                { "task": "Import data", "type": "....." }
                        ]
                },
"""

As last task, close the selection block ```if``` adding the proper depencies.

In [ ]:
workflow_if_endif_else += """
                {
                        "name": "Selection block end",
                        "operator": "oph_endif",
                        "arguments": [ ],
                        "dependencies":
                        [
                                { "task": "....."},
                                { "task": "....." },
                                { "task": "....." }
                        ]
                }
        ]
}
"""

And now, if everything is fine, we are ready to run the workflow with different input parameters to test the three possible branches!

**First case**: new datacube randomly generated inside a container named "workflow"
    
 <img src="../imgs/Selection_Interface_randcube.svg" alt="Selection_Interface_randcube" width="200">
 
**Second case**: datacube first imported, then subsetted

<img src="../imgs/Selection_Interface_import+subset.svg" alt="Selection_Interface_import+subset" width="200">

**Third case**: datacube imported and subsetted at the same time
    
 <img src="../imgs/Selection_Interface_importANDsubset.svg" alt="Selection_Interface_importANDsubset" width="200">

Define the workflow arguments:

In [ ]:
# Try all the possible options!
if_cond=...
elseif_cond=...

file="/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable="tasmax"

# Try yourself with different filters!
lat_lon_time="-50:10|20:140|150:240" 

Run the workflow specifing the arguments in the proper order.

In [ ]:
cube.Cube.client.wsubmit(workflow_if_endif_else, .....)

Check your workspace

In [ ]:
cube.Cube.list(level=2)

Empty your workspace before running the workflow again

In [ ]:
cube.Cube.deletecontainer(container="random",force='yes')
cube.Cube.deletecontainer(container="tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc",force='yes')

**NOTE:** You can find a sample implementation of this exercise in the *Examples* folder in the notebook [**Selection_interface**](../Examples/Selection_interface_example.ipynb)